In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
t=pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
te=pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

In [ ]:
t.head()

In [ ]:
te.head()

In [ ]:
x=t['Category'].value_counts()
x.shape

In [ ]:
t.shape

In [ ]:
te.shape

In [ ]:
t.info()

In [ ]:
te.info()

In [ ]:
t.drop(['Descript','Resolution','Address'],inplace=True,axis=1)
te.drop('Address',axis=1,inplace=True)

t['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)
te['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)




In [ ]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
te['DayOfWeek']=l.fit_transform(te['DayOfWeek'])
te.head()

In [ ]:
t.head()

In [ ]:
te.head()

In [ ]:
X=t.drop(['Category'],axis=1)
y=pd.get_dummies(t['Category'])

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)



In [ ]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
te['DayOfWeek']=l.fit_transform(te['DayOfWeek'])
te.head()

In [ ]:
t.info()

In [ ]:
te.info()

In [ ]:
t['Category'].value_counts()

In [ ]:
type(t['Dates'])

In [ ]:
_ = sns.heatmap(t.corr(), annot = True, fmt = ".2f", cmap = 'YlGnBu')

In [ ]:
t.hist(column=t.columns, figsize=(20,7),layout=(1,13))

In [ ]:
ls1=list(t['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][:4:]
t['Year']=ls1 

ls1=list(t['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][5:7:]
t['Month']=ls1 

ls1=list(t['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][8:10:]
t['Day']=ls1 

ls1=list(t['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-6:]
t['Hours']=ls1

ls1=list(t['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][-5:-3:]
t['Minutes']=ls1

t.drop('Dates',axis=1,inplace=True)
t=pd.concat([t,pd.get_dummies(t['PdDistrict'])],axis=1)
t.drop('PdDistrict',axis=1,inplace=True)
t.head()

In [ ]:
ls1=list(te['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][:4:]
te['Year']=ls1 

ls1=list(te['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][5:7:]
te['Month']=ls1 

ls1=list(te['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][8:10:]
te['Day']=ls1 

ls1=list(te['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-6:]
te['Hours']=ls1

ls1=list(te['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][-5:-3:]
te['Minutes']=ls1

te.drop('Dates',axis=1,inplace=True)
te=pd.concat([te,pd.get_dummies(te['PdDistrict'])],axis=1)
te.drop('PdDistrict',axis=1,inplace=True)
te.head()

In [ ]:
X=t.drop(['Category'],axis=1)
X=X.astype(float)
y=pd.get_dummies(t['Category'])
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
y.dtypes

In [ ]:
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense,Activation

model=Sequential()
model.add(Dense(128,input_shape=(X.shape[1],)))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(39,activation='softmax'))
model.summary()



In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

train=model.fit(X_train,y_train, 
         batch_size=32,
         epochs=10,
         verbose=2,
         validation_data=(X_test,y_test))



In [ ]:
test=te.drop(['Id'],axis=1)
test=test.astype(float)
test.dtypes

In [ ]:
pred=model.predict(test)

m = np.max(pred, axis=1).reshape(-1, 1)
predicted = np.array((pred == m), dtype='int32')
predicted



In [ ]:
sample=pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
col_names=list(sample.columns)
col_names.remove('Id')
print(col_names)

In [ ]:
submission = pd.DataFrame()
submission['Id']=te['Id']
for i, entry in enumerate(col_names):
    submission[entry] = predicted[:,i]

submission.head()



In [ ]:
submission.to_csv('../working/submission.csv', index=False)